In [1]:
import numpy as np
import random

In [2]:
boardh = 9
boardw = 9
board = np.array([
                  [0,0,3,0,2,0,6,0,0],
                  [9,0,0,3,0,5,0,0,1],
                  [0,0,1,8,0,6,4,0,0],
                  [0,0,8,1,0,2,9,0,0],
                  [7,0,0,0,0,0,0,0,8],
                  [0,0,6,7,0,8,2,0,0],
                  [0,0,2,6,0,9,5,0,0],
                  [8,0,0,2,0,3,0,0,9],
                  [0,0,5,0,1,0,3,0,0]
                 ])



In [3]:
def printBoard(board):
    for r in range(board.shape[0]):
        for c in range(board.shape[1]):
            print(board[r,c], end=" ")
            if (c+1)%3 == 0 and c+1 != board.shape[1]:
                print("| ", end="")
            if c+1 == board.shape[1]:
                print("\r")
        if (r+1)%3 == 0 and r+1 != board.shape[0] :
            print("---------------------")
    print("")
    return

In [4]:
def elimateOutcome(candidate, vector):
    for i in range(len(vector)):
        if vector[i] != 0:
            index = vector[i]-1
            candidate[index] = 0
    return candidate

In [5]:
def returnGridMatrix(board, r, c):
    rstart = (int((r)/3))*3
    rend = (int((r)/3))*3 + 3
    cstart = (int((c)/3))*3
    cend = (int((c)/3))*3 + 3
    grid = board[rstart:rend,cstart:cend]
    return grid

In [6]:
def findCandidates(board):
    
    candidates = np.ones([9,9,9])
    
    for r in range(board.shape[0]):
        for c in range(board.shape[1]):
            if board[r,c] == 0:
                candidates[r,c,:] = elimateOutcome(candidates[r,c,:], board[r,:])
                candidates[r,c,:] = elimateOutcome(candidates[r,c,:], board[:,c].T)
                grid = returnGridMatrix(board, r, c).reshape(1,-1)
                candidates[r,c,:] = elimateOutcome(candidates[r,c,:], grid.T)
            else:
                candidates[r,c,:] = 0
    return candidates

In [7]:
def isSolved(board_updated):
    solveStatus = np.sum(np.sum(board_updated, axis=0) - np.sum(board_updated, axis=1)) == 9
    if solveStatus == True:
        print("Board Solved!")
        printBoard(board_updated)
    else:
        print(str(np.sum(board_updated==0)) + " unsolved boxes")
    return solveStatus

In [8]:
def fillOnlyOnePossibleSlot(board_updated, candidates):
    this_board = board_updated
    for r in range(board_updated.shape[0]):
        for c in range(board_updated.shape[1]):
            if np.sum(candidates[r,c,:]) == 1:
                this_board[r,c] = np.argmax(candidates[r,c,:])+1
    return board_updated

In [9]:
def findUniqueCandidate(board_updated, candidates):
    temp_board = board_updated
    
    for r in range(board_updated.shape[0]):
    #for r in range(4,5):
        crosscol_occ = np.sum(candidates[r,:], axis=0)
        #print(candidates[r,:])
        #print(crosscol_occ)
        pos = np.where(crosscol_occ==1)
        noOfUnique_occ = pos[0].shape[0]
        if noOfUnique_occ > 0: 
            for p in pos[0]:
                c1 = np.squeeze(np.where(candidates[r,:][:,p] == 1))
                #print(r,c1,p)
                #if temp_board[r,c1] == 0:
                #print(candidates[r,c1])
                temp_board[r,c1] = p+1
                candidates[r,c1][p] = 0
                
    for c in range(board_updated.shape[1]):
    #for r in range(4,5):
        crosscol_occ = np.sum(candidates[:,c], axis=0)
        #print(candidates[r,:])
        print(crosscol_occ)
        pos = np.where(crosscol_occ==1)
        noOfUnique_occ = pos[0].shape[0]
        if noOfUnique_occ > 0: 
            for p in pos[0]:
                r1 = np.squeeze(np.where(candidates[:,c][:,p] == 1))
                #print(r,c1,p)
                #if temp_board[r,c1] == 0:
                #print(candidates[r,c1])
                temp_board[r1,c] = p+1
                candidates[r1,c][p] = 0
                
    return temp_board, candidates

In [10]:
def solve(board):
    onlyOneCandAvail = True
    count = 0
    board_updated = board
    while onlyOneCandAvail == True:
        candidates = findCandidates(board_updated)
        board_updated = fillOnlyOnePossibleSlot(board_updated, candidates)
        count += 1
        if np.sum(board_updated == 0) == 0 or count > 9:
            onlyOneCandAvail = False
    if not isSolved(board_updated):
        board_updated, candidates = findUniqueCandidate(board_updated, candidates)
        #board_updated.T = findUniqueCandidate(board_updated.T, candidates.T)
    return board_updated, candidates

In [11]:
board_updated = np.copy(board)
board_updated, candidates = solve(board_updated)
printBoard(board_updated)

0 unsolved boxes
[ 0.  0.  0.  0.  1.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  1.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
4 8 3 | 9 2 1 | 6 5 7 
9 6 7 | 3 4 5 | 8 2 1 
2 5 1 | 8 7 6 | 4 9 3 
---------------------
5 4 8 | 1 3 2 | 9 7 6 
7 2 9 | 5 6 4 | 1 3 8 
1 3 6 | 7 9 8 | 2 4 5 
---------------------
3 7 2 | 6 8 9 | 5 1 4 
8 1 4 | 2 5 3 | 7 6 9 
6 9 5 | 4 1 7 | 3 8 2 



In [12]:
def cross(a, b):
    return [s+t for s in a for t in b]

In [13]:
rows = 'ABCDEFGHI'
cols = '123456789'
grid = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
grid2 = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
boxes = cross(rows, cols)

row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
unitlist = row_units + column_units + square_units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)


In [14]:
def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

In [15]:
def grid_values(grid):
    
    grid_dict = {}
    dpossibleVals = '123456789'
    
    row_units = [cross(r, cols) for r in rows]
    
    for r in range(len(row_units)):
        for c in range(len(row_units[1])):
            if grid[r*9+c] != '.':
                grid_dict[row_units[r][c]] = grid[r*9+c]
            else:
                grid_dict[row_units[r][c]] = dpossibleVals
    
    return grid_dict

In [16]:
def eliminate(values):
    for key, val in values.items():
        if len(val) == 1:
            for u in range(3):
                for b in units[key][u]:
                
                    if val in values[b] and val != values[b]:
                        values[b] = values[b].replace(val, '')
    return values

In [17]:
def only_choice(values):
    
    """
    for units in (row_units, column_units, square_units):
        for sq in units:
            for d in range(len(cols)):
                current_d_count = 0
                current_digit = d+1
            
                for u in sq:
                    if str(current_digit) in str(values[u]) and str(current_digit) != values[u]:
                        current_d_count += 1
                        last_unit = u
                if current_d_count == 1:
                    values[last_unit] = str(current_digit)
    """ 
    tmp_values = dict(values)
    
    for box, nhb_units_array in units.items():

        possible_values = tmp_values[box]
        #print(box)
        
        if len(possible_values) > 1: # not consider those assigned
            for poss_val_index in range(len(possible_values)):
                
                this_val = possible_values[poss_val_index]
                
                
                
                for nhb_units_row in nhb_units_array:  #nhb_units_array has len of 3
                    
                    count = 0
                    
                    for u in nhb_units_row:
                        #print(u)
                        
                        if this_val in tmp_values[u] and box != u:
                            count += 1
                            
                    if count == 0:
                        #print(box, this_val)
                        tmp_values[box] = this_val
                        
    return tmp_values


In [18]:
def reduce_puzzle(values):
    stalled = False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])

        # Your code here: Use the Eliminate Strategy
        values = eliminate(values)
        
        # Your code here: Use the Only Choice Strategy
        values = only_choice(values)
        
        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

In [19]:
def isSolved(values):
    
    solvedStatus = True
    
    for unit in (column_units, row_units, square_units):
        for arr in unit:
            sub_total = 0
            for u in arr:
                sub_total += int(values[u])
            
            if sub_total != 45:
                solvedStatus = False
    
    
    return solvedStatus

In [36]:
def search(values, recur = False):
    "Using depth-first search and propagation, create a search tree and solve the sudoku."
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)
    
    # Choose one of the unfilled squares with the fewest possibilities
    
    if isSolved(values) == False:
        box_fewest_poss = ''
        min_length = 9
        for r in row_units:
            for u in r:
                if len(values[u]) != 1 and len(values[u]) < min_length:
                    box_fewest_poss = u
                    min_length = len(values[u])
        #display(values)
        if box_fewest_poss != '':
            for pos_val in values[box_fewest_poss]:
                #print(box_fewest_poss, values[box_fewest_poss], pos_val)
                if recur == True:
                    values[box_fewest_poss] = pos_val
                    values = search(values, True)
        else:
            #print(box_fewest_poss)
            display(values)
            
    
    # Now use recursion to solve each one of the resulting sudokus, 
    # and if one returns a value (not False), return that answer!

    # If you're stuck, see the solution.py tab!
    return values

In [37]:
grid_dict1 = grid_values(grid2)
#display(grid_dict2)
grid_dict2 = search(grid_dict1, True)
print(isSolved(grid_dict2))
#display(grid_dict2)

4 1 7 |3 6 9 |8 2 5 
6 3 2 |1 5 8 |7 9 4 
9 5 8 |7 2 4 |6 1 3 
------+------+------
8 2 5 |4 3 7 |1 6 9 
7 9 1 |5 8 6 |4 5 2 
7 4 6 |9 1 2 |3 8 7 
------+------+------
2 8 9 |6 4 3 |5 7 1 
5 6 3 |2 7 1 |9 4 8 
1 7 4 |8 9 5 |2 3 6 
4 1 7 |3 9 9 |8 2 5 
6 3 2 |1 5 8 |7 9 4 
9 5 8 |7 2 4 |6 1 3 
------+------+------
8 2 5 |4 3 7 |1 6 9 
7 9 1 |5 8 6 |4 5 2 
7 4 6 |9 1 2 |3 8 7 
------+------+------
2 8 9 |6 4 3 |5 7 1 
5 6 3 |2 7 1 |9 4 8 
1 7 4 |8 9 5 |2 3 6 
4 1 7 |3 9 9 |8 9 5 
6 3 2 |1 5 8 |7 9 4 
9 5 8 |7 2 4 |6 1 3 
------+------+------
8 2 5 |4 3 7 |1 6 9 
7 9 1 |5 8 6 |4 5 2 
7 4 6 |9 1 2 |3 8 7 
------+------+------
2 8 9 |6 4 3 |5 7 1 
5 6 3 |2 7 1 |9 4 8 
1 7 4 |8 9 5 |2 3 6 
4 1 7 |9 9 9 |8 9 5 
6 3 2 |1 5 8 |7 9 4 
9 5 8 |7 2 4 |6 1 3 
------+------+------
8 2 5 |4 3 7 |1 6 9 
7 9 1 |5 8 6 |4 5 2 
7 4 6 |9 1 2 |3 8 7 
------+------+------
2 8 9 |6 4 3 |5 7 1 
5 6 3 |2 7 1 |9 4 8 
1 7 4 |8 9 5 |2 3 6 
4 9 7 |9 9 9 |8 9 5 
6 3 2 |1 5 8 |7 9 4 
9 5 8 |7 2 4 |6 1 3 
------+------